In [ ]:
# %pip install pandas

In [ ]:
import psycopg2 as ps
import pandas as pd

In [ ]:
conn = ps.connect(dbname='postgres', user='student', 
                        password='student', host='104.197.87.141')


In [ ]:
df = pd.read_sql_query("SELECT surname, name, absences, unready_labs FROM faculty_ua.students", con=conn)
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.iloc[3,1]

In [ ]:
df1 = df[df['absences']>110]
df1

In [ ]:
df1 = df[df['absences']>110]

df1.style.highlight_max(subset = ['absences'], color='red', axis = 0) \
    .highlight_min(subset = ['absences'], color='green', axis = 0) \
    .highlight_max(subset = ['unready_labs'], color='magenta', axis = 0) \
    .highlight_min(subset = ['unready_labs'], color='lightgreen', axis = 0)

In [ ]:
df1.sort_values('absences', ascending=False).style.bar(subset=['absences'], color='darkred')#, vmin=110)

In [ ]:
dfg = pd.read_sql_query(
    "SELECT g.group_number, COUNT(s.student_id) students, SUM(s.absences) absences, sum(unready_labs) unready_labs "
    + "FROM faculty_ua.students s JOIN faculty_ua.groups g "
    + "ON s.group_id = g.group_id "
    + "GROUP BY g.group_number "
    , con=conn)
dfg

In [ ]:
dfg.sort_values('students').plot(x = 'students', y = 'absences')

In [ ]:
dfg.plot.scatter(x = 'students', y = 'absences')

In [ ]:
 # %pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 12), dpi= 80)
# fig, ax = plt.subplots()

grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.2)
ax = fig.add_subplot(grid[:-1, :-1])

x = list(dfg['students'])
y = list(dfg['absences'])
sizes = list(dfg['unready_labs'])

cources = [str[1] for str in list(dfg['group_number'])]
colors = {1:'indigo', 2:'coral', 3:'blue', 4:'lime', 5:'magenta'}
cololor_list =  [colors[int(cr)] for cr in cources]

# i=0;
# while i < len(cololor_list):
#     print(list(dfg['group_number'])[i], cources[i],  cololor_list[i])
#     i+=1

scatter = ax.scatter(x, y, c=cololor_list, s=sizes, label=colors)
ax.set_title("Пропуски занять у групах")
ax.legend()
handles, labels = scatter.legend_elements(prop="sizes")
legend1 = ax.legend(handles, labels, loc="upper right", title="Заборг. з ЛР")
ax.set_xlabel('кількість студентів', fontsize = 12)
ax.set_ylabel('пропуски', fontsize = 12)

# бокові гістограми

# ax_right = fig.add_subplot(grid[:-1, -1], xticklabels=[], yticklabels=[])
# ax_bottom = fig.add_subplot(grid[-1, 0:-1], xticklabels=[], yticklabels=[])
# ax_bottom.hist(x, len(x), histtype='stepfilled', orientation='vertical', color='mediumvioletred')
# ax_bottom.invert_yaxis()
# ax_right.hist(y, len(y), histtype='stepfilled', orientation='horizontal', color='mediumvioletred')

plt.show()


In [ ]:
i=0;
while i < len(cololor_list):
    print(list(dfg['group_number'])[i], cources[i],  cololor_list[i])
    i+=1

In [ ]:
dfcc = pd.read_sql_query(
    "SELECT c.chair_number AS chair, SUBSTRING(g.group_number, 2, 1) AS course, "
    + "COUNT(s.student_id) students, SUM(s.absences) absences, sum(unready_labs) unready_labs "
    + "FROM faculty_ua.students s JOIN faculty_ua.groups g ON s.group_id = g.group_id "
	+ "JOIN faculty_ua.chairs c ON c.chair_id = g.chair_id "
    + "GROUP BY c.chair_number, course "
    , con=conn)
dfcc

In [ ]:
import matplotlib.pyplot as plt


dfs = dfcc.sort_values('chair')

x = list(dfs['chair'])
y = list(dfs['course'])
z = list(dfs['absences'])

plt.figure(figsize=(16,10), dpi= 80)
figg, axx = plt.subplots(subplot_kw={"projection": "3d"})

axx.plot_trisurf(x, y, z,cmap='magma')

axx.set_xlabel('кафедри')
axx.set_ylabel('курси')
axx.set_zlabel('пропуски')

plt.show()

In [ ]:
# %pip install seaborn --user

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

chart = dfcc.pivot('chair', 'course', 'absences')

f = plt.subplots()
sns.heatmap(chart, annot=True, linewidths=.5, fmt="d", cmap="magma")

In [ ]:
import numpy as np

dfsum = dfcc.groupby(by='chair', group_keys=True).sum()

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), subplot_kw=dict(aspect="equal"))
fig.suptitle('Досягнення кафедр', fontfamily = 'serif',
                fontstyle = 'italic',
                fontsize = 15)
fig.set_facecolor('thistle') 

size = 0.4
chairs = list(dfcc.groupby(by='chair', group_keys=True).groups.keys())

# перший пончик

abs_outer = list(dfsum['absences'])
abs_inner = list(dfcc.sort_values(by=['chair', 'course'])['absences'])

clr_outer1 = plt.cm.autumn(np.linspace(0,1,len(abs_outer)))
clr_inner1 = plt.cm.autumn(np.linspace(0,1,len(abs_inner)))

ax[0].pie(abs_outer, startangle=0, pctdistance =0.88, colors=clr_outer1, labels=chairs,
       autopct = '%1.1f%%', labeldistance=1.05, radius= 1.0, wedgeprops=dict(width=0.95*size, edgecolor='w'), shadow=True,
       textprops=dict(color='blue', size=12, weight='bold'))

ax[0].pie(abs_inner, startangle=0, pctdistance =0.88, colors=clr_inner1,
        radius= 1-size, wedgeprops=dict(width=size, edgecolor='w'), shadow=True)

ax[0].set_title("Пропуски занять по кафедрам")

# другий пончик

st_outer = list(dfsum['students'])
st_inner = list(dfcc.sort_values(by=['chair', 'course'])['students'])

clr_outer2 = plt.cm.winter(np.linspace(0,1,len(st_outer)))
clr_inner2 = plt.cm.winter(np.linspace(0,1,len(st_inner)))

ax[1].pie(st_outer, startangle=0, pctdistance =0.88, colors=clr_outer2, labels=chairs,
       autopct = '%1.1f%%', labeldistance=1.05, radius= 1.0, wedgeprops=dict(width=0.95*size, edgecolor='w'), shadow=True,
       textprops=dict(color='red', size=12, weight='bold'))

ax[1].pie(st_inner, startangle=0, pctdistance =0.88, colors=clr_inner2,
        radius= 1-size, wedgeprops=dict(width=size, edgecolor='w'), shadow=True)

ax[1].set_title("Кількість студентів занять по кафедрам")

plt.show()